In [1]:
import os
import pandas as pd 
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'AppleGothic' # 폰트 변경
plt.rcParams['axes.unicode_minus'] = False # 축 값 마이너스 깨짐 해결
import numpy as np
from torch.nn import Transformer
from torch import nn
import torch
import math
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_absolute_percentage_error ,mean_squared_error

In [9]:
#path = '~/workplace/datas/energy/'
path = '~/Desktop/workplace/datas/energy/'

In [10]:
train_df = pd.read_csv(path + 'encoded_train_df.csv', index_col=0)
test_df = pd.read_csv(path + 'encoded_test_df.csv', index_col=0)

In [11]:
X = train_df.iloc[:, :-1]
y = train_df.iloc[:, -1]

In [12]:
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]

In [13]:
X.shape, y.shape, X_test.shape, y_test.shape

((546336, 13), (546336,), (2016, 13), (2016,))

In [14]:
#전력수요 예측 범위에 따라 초단기, 단기, 중기, 장기로 구분하고 기상데이터 등 추가 정보를 활용하는 방안
#모델 선정 고려사항 - 데이터 수집 기간, 예측 대상 기간과 지역, 전력수요 데이터 및 추가 데이터 사용 여부등


#기온 데이터를 반영해 예측 시점 기준 24시간 이후에 발생하는 (1시간 단위)
#24시간 동안의 전력수요를  예측 ===>> 즉 하루예측


#엄밀한 모델구축을 위해서는 수요발생 시점의 기온 예측값을 사용하는 것이 타당하지만,
#기온 예측값을 수집하는 것이 불가능하여 실제 기온 측정값을 사용



# WaveNet 모델 구축

In [15]:
#WFV (walk forward validation) 검증 기법 사용


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('현재 디바이스:', device)


현재 디바이스: cpu


# DataLoader

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader



# numpy array를 torch tensor로 변환
X_train = torch.tensor(X.to_numpy(), dtype=torch.float32)
y_train = torch.tensor(y.to_numpy().reshape(-1,1), dtype=torch.float32)
X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test = torch.tensor(y_test.to_numpy().reshape(-1, 1), dtype=torch.float32)

#dataset 만들기
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

#dataloader 
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [24]:
X_train.shape, y_train.shape, X_test.shape , y_test.shape

(torch.Size([546336, 13]),
 torch.Size([546336, 1]),
 torch.Size([2016, 13]),
 torch.Size([2016, 1]))

In [23]:
X_train.dtype, y_train.dtype

(torch.float32, torch.float32)

In [65]:
for x, y in train_loader:
    print(x.unsqueeze(2).shape, y.shape)

#사이즈가 32, 13 // 32,1  (batch_size, feature)

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Size([64, 1])
torch.Size([64, 13, 1]) torch.Si

# Model building

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim

In [56]:
#파라미터 값
batch_size = 32
sequence_length = X_train.shape[1]

In [76]:
a = nn.Sequential(nn.Conv1d(13, 32, 1))(x)
a.shape

RuntimeError: Given groups=1, weight of size [32, 13, 1], expected input[1, 32, 13] to have 13 channels, but got 32 channels instead

In [71]:
class DilatedCausalConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, bias):
        super(DilatedCausalConv1d, self).__init__()
        self.dilation = dilation
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size, 
                                padding=dilation*(kernel_size-1),
                                dilation=dilation, bias=bias)
    
        #13, 32
    
    def forward(self, x):
        return self.conv1d(x)[:, :, :-self.dilation] 
    
    #dilation이 잘 이해가 안가네..
    
    
class ResidualBlock(nn.Module):
    def __init__(self, res_channels, skip_channels, dilation):
        super(ResidualBlock, self).__init__()
        self.conv = DilatedCausalConv1d(res_channels, 2*res_channels,
                                       kernel_size=2, dilation=dilation, bias=True)
        self.res_conv = nn.Conv1d(res_channels, res_channels, 1)
        self.skip_conv = nn.Conv1d(res_channels, skip_channels, 1)
        
    def forward(self, x):
        output = self.conv(x)
        tanh_out, sigmoid_out = output.chunk(2, dim=1) #chunk 두개로 쪼개는 함수
        output = torch.tanh(tanh_out) * torch.sigmoid(sigmoid_out)
        
        res_out = self.res_conv(output)
        skip_out = self.skip_conv(output)
        
        
        return res_out + x, skip_out
    
    
    
class WaveNet(nn.Module):
    def __init__(self, layer_size, stack_size, in_channels, res_channels, skip_channels):
        super(WaveNet, self).__init__()
        self.causal_conv = DilatedCausalConv1d(in_channels, res_channels, 1, dilation=1, bias=True)
        self.res_blocks = nn.ModuleList([ResidualBlock(res_channels, skip_channels, dilation=2**i) for i in range(layer_size)] * stack_size)
        self.conv_out = nn.Sequential(
            nn.ReLU(),
            nn.Conv1d(skip_channels, skip_channels, 1),
            nn.ReLU(),
            nn.Conv1d(skip_channels, out_channels=1, kernel_size=1)
        )
        
    
    def forward(self, x):
        output = self.causal_conv(x)
        skip_conntections = []
        
        for res_block in self.res_blocks:
            otuput, skip_out = res_block(output)
            skip_conntections.append(skip_out)
            
        output = sum(skip_conntections)
        return self.conv_out(output)
        

#parameters
epochs = 1


model = WaveNet(layer_size=4, stack_size=3, 
                in_channels=sequence_length, #13
                res_channels=32,
               skip_channels=32)


#손실 함수, 활성화 함수
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

#train
for epoch in range(epochs):

    for i, (inputs, target) in enumerate(train_loader):
        #차원조정
        #torch.Size([64, 13]) torch.Size([64, 1])
        
        inputs = inputs.view(-1, sequence_length, 1)
        target = target.view(-1, 1)


        #입력 및 손실 계산
        outputs = model(inputs)
        loss = criterion(outputs, target)
        
        #역전파 & 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    

RuntimeError: Only zero batch or zero channel inputs are supported, but got input shape: [64, 32, 1, 0]

In [64]:
outputs.shape, target.shape

(torch.Size([1, 1, 31]), torch.Size([32, 1]))

In [ ]:
#평가모드

from torchmetrics.regression import MeanAbsolutePercentageError

model.eval()



with torch.no_grad():
    total_loss = 0
    for i, (inputs, target) in enumerate(test_loader):
        
        #차원조정
        inputs = inputs.view(1,sequence_length, -1)
        target = target.view(-1, 1)
        
        
        #모델에 입력 및 손실 계산
        outputs = model(inputs)
        loss = criterion(outputs, target)
        total_loss += loss.item()
        
        
        #예측성능검증
        metric = MeanAbsolutePercentageError()
        mape = metric(outputs, target)
        
        
    # 평균 손실 출력
    print(f'Test Loss: {total_loss / len(test_loader)} mape:{mape}')
        

In [ ]:
# 예측할 시퀀스 데이터 설정
sequence = X_test[0]  # X_test의 첫 번째 시퀀스를 예로 듭니다

# 모델을 평가 모드로 설정
model.eval()

with torch.no_grad():  # 예측 과정에서는 기울기 계산이 필요 없습니다
    # 차원 조정 및 모델에 입력
    inputs = sequence.view(-1, sequence_length, 1)
    prediction = model(inputs)

    # 예측 결과 출력
    print(f'Prediction: {prediction.item()}')
